In [1]:
import pyiron_workflow as pwf

In [2]:
@pwf.as_inp_dataclass_node
class MyInpData:
    question: str = "still computing"
    answer: int = 42

@pwf.as_out_dataclass_node
class MyOutData:
    question: str = "still computing"
    answer: int = 42

wf = pwf.Workflow("auto_encoding_graph")
wf.to_dataclass = MyInpData("input will be output", 0)
wf.from_dataclass = MyOutData(wf.to_dataclass)
wf.run()

('input will be output', 0)

In [3]:
@pwf.as_macro_node(["q", "a"])
def auto_encoder(q: str, a: int):
    wf = pwf.Workflow("auto_encoding_subgraph")
    wf.to_dataclass = MyInpData(q, a)
    wf.from_dataclass = MyOutData(wf.to_dataclass)
    return wf.from_dataclass.outputs.question, wf.from_dataclass.outputs.answer

macro = auto_encoder()
macro("also when it's a macro", 1)

("also when it's a macro", 1)

It's hidden behind a private variable, but we can investigate the internal state of the subgraph -- i.e. the macro object contains its subgraph's retrospective provenance

In [4]:
macro._wf_macro.to_dataclass.outputs.dataclass.value

MyInpData(question="also when it's a macro", answer=1)

In [5]:
import pyiron_nodes as pn

wf2 = pwf.Workflow("my_workflow")
wf2.to_dataclass = pn.liam.MyInpData("provenance is great", 2)
wf2.from_dataclass = pn.liam.MyOutData(wf2.to_dataclass.outputs.dataclass)
wf2.macro = pn.liam.auto_encoder(wf2.from_dataclass.outputs.question, wf2.from_dataclass.outputs.answer)
wf2.run()

('provenance is great', 2)

`Workflow` objects are already inspectable, since we can always look at their children:

In [6]:
wf = pwf.Workflow("my_workflow")
wf.to_dataclass2 = MyInpData("provenance is great", 2)
wf.from_dataclass = MyOutData(wf.to_dataclass2) # .outputs.dataclass)
wf.macro = auto_encoder(wf.from_dataclass.outputs.question, wf.from_dataclass.outputs.answer)
wf.run()

('provenance is great', 2)

In [7]:
wf.macro._wf_macro.to_dataclass.outputs.dataclass.value

MyInpData(question='provenance is great', answer=2)

And we can nest macros, maintaining provenance to arbitrary depth

In [8]:
@pwf.as_macro_node(["deep_q", "deep_a"])
def nested_macro(q: str, a: int):
    wf = pwf.Workflow("nested_macro_subgraph")
    wf.to_dataclass = MyInpData(q, a)
    wf.from_dataclass = MyOutData(wf.to_dataclass)
    wf.sub_macro = auto_encoder(wf.from_dataclass.outputs.question, wf.from_dataclass.outputs.answer)
    return wf.sub_macro.outputs.q, wf.sub_macro.outputs.a

nm = nested_macro()
nm("we might want to make this optional so the garbage collector can save us memory", 3)

('we might want to make this optional so the garbage collector can save us memory',
 3)

In [9]:
nm._wf_macro.sub_macro._wf_macro.to_dataclass.outputs.dataclass.value

MyInpData(question='we might want to make this optional so the garbage collector can save us memory', answer=3)

In [10]:
wf

pyiron_workflow instance: 
from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("my_workflow")

wf.to_dataclass2 = __main__.MyInpData(question="provenance is great", answer=2)
wf.from_dataclass = __main__.MyOutData(dataclass=wf.to_dataclass2)
wf.macro = __main__.auto_encoder(
    q=wf.from_dataclass.outputs.question,
    a=wf.from_dataclass.outputs.answer,
    q="provenance is great",
    a=2,
)

In [11]:
wf.macro.pull()

('provenance is great', 2)

In [12]:
from pyiron_workflow.graph import base

q = "question"
a = "answer"
wf = pwf.Workflow("auto_encoding_subgraph")
wf.to_dataclass = MyInpData(q, a)
wf.from_dataclass = MyOutData(wf.to_dataclass)

In [13]:
graph = base.get_full_graph_from_wf(wf2)

Adding macro node macro
adding graph node:  macro False
full graph Adding macro node macro False


In [14]:
graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,to_dataclass,pyiron_nodes.liam.MyInpData,to_dataclass,macro,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,from_dataclass,pyiron_workflow.simple_workflow.func_dataclass,from_dataclass,macro,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,macro,pyiron_nodes.liam.auto_encoder,macro,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='macro', label='macro', root_node=Non...",graph,customNode,False
3,va_i_macro__q,pyiron_workflow.graph.base.identity,va_i_macro__q,macro,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,va_i_macro__a,pyiron_workflow.graph.base.identity,va_i_macro__a,macro,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,va_o_macro__q,pyiron_workflow.graph.base.identity,va_o_macro__q,macro,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,va_o_macro__a,pyiron_workflow.graph.base.identity,va_o_macro__a,macro,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [15]:
graph.edges

,source,target,sourceHandle,targetHandle
0,va_i_macro__q,to_dataclass,x,question
1,va_i_macro__a,to_dataclass,x,answer
2,to_dataclass,from_dataclass,dataclass,dataclass
3,from_dataclass,va_o_macro__q,question,x
4,from_dataclass,va_o_macro__a,answer,x
5,to_dataclass,from_dataclass,dataclass,dataclass
6,from_dataclass,va_i_macro__q,question,x
7,from_dataclass,va_i_macro__a,answer,x


In [16]:
base.GuiGraph(graph)

ReactFlowWidget(layout=Layout(height='600px', width='800px'))

In [17]:
from pyiron_workflow.graph.gui import PyironFlow

widget = PyironFlow([wf2, 'encoder_test'])
widget.gui

Adding macro node macro
adding graph node:  macro False
full graph Adding macro node macro False


In [18]:
graph = base.GuiGraph(base.get_graph_from_macro_node(auto_encoder()))

In [19]:
widget = PyironFlow([wf])
widget.gui

Adding macro node macro
adding graph node:  macro False
full graph Adding macro node macro False


In [20]:
graph_encoder = base._load_graph("encoder_test")

print(base.graph_to_code(graph_encoder))


def encoder_test():

    from pyiron_workflow import Workflow
    import pyiron_nodes

    wf = Workflow('encoder_test')

    wf.MyInpData = pyiron_nodes.liam.MyInpData() 
    wf.MyOutData = pyiron_workflow.simple_workflow.func_dataclass(dataclass=wf.MyInpData) 
    wf.auto_encoder = pyiron_nodes.liam.auto_encoder(q=wf.MyOutData.outputs.question, a=wf.MyOutData.outputs.answer) 
    wf.nested_macro = pyiron_nodes.liam.nested_macro(a=wf.MyOutData.outputs.answer, q=wf.MyOutData.outputs.question) 

    return 



In [21]:
from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow('encoder_test')

wf.toDataclass = pyiron_nodes.liam.MyInpData("provenance is great", 2) 
wf.MyOutData = pyiron_nodes.liam.MyOutData(dataclass=wf.toDataclass) 
wf.auto_encoder = pyiron_nodes.liam.auto_encoder(q=wf.MyOutData.outputs.question, a=wf.MyOutData.outputs.answer) 
# wf.nested_macro = pyiron_nodes.liam.nested_macro(a=wf.MyOutData.outputs.answer, q=wf.MyOutData.outputs.question) 

In [22]:
wf = pwf.Workflow("my_workflow")
wf.to_dataclass2 = MyInpData("provenance is great", 2)
wf.from_dataclass2 = MyOutData(wf.to_dataclass2.outputs.dataclass)
wf.macro = auto_encoder(wf.from_dataclass2.outputs.question, wf.from_dataclass2.outputs.answer)
wf.run()

('provenance is great', 2)

In [23]:
widget = PyironFlow([wf])
widget.gui

Adding macro node macro
adding graph node:  macro False
full graph Adding macro node macro False


Expanding node nested_macro
Expanding node auto_encoder
